In [1]:
import itertools
import os
import numpy as np
import pandas as pd
import requests

In [13]:
aus_suburbs = pd.read_csv(os.path.join('aus_suburbs.csv'), index_col=0)

In [15]:
aus_suburbs = aus_suburbs.drop(['State',
                                'Population',
                                'distance',
                                'geometry',
                                'area',
                                'density'], axis=1)

# Search for local venues for each every suburb with Foursquare API

### Define Foursquare credentials and version

In [3]:
FOURSQUARE_CLIENT_ID = 'JVA5H5NJXBQTUPSMHXXD0V0NKNP0OVJO0GKU1WJGLER5Q0DU'
FOURSQUARE_SECRET = 'DMZV42OBBRZNSPQSGEUD3PE3N5EHUKRZYRTMSCLSORKPAO2W'
FOURSQUARE_API_VERSION = '20180605'

In [4]:
CLIENT_ID = FOURSQUARE_CLIENT_ID # Foursquare ID
CLIENT_SECRET = FOURSQUARE_SECRET # Foursquare Secret
VERSION = FOURSQUARE_API_VERSION # Foursquare API version

print('Credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Credentails:
CLIENT_ID: JVA5H5NJXBQTUPSMHXXD0V0NKNP0OVJO0GKU1WJGLER5Q0DU
CLIENT_SECRET:DMZV42OBBRZNSPQSGEUD3PE3N5EHUKRZYRTMSCLSORKPAO2W


In [21]:
def getNearbyVenues(suburb_id, names, postcode, lga, city, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    index = 0
    for suburb_id, name, postcode, lga, city, lat, lng in zip(suburb_id, names, postcode, lga, city, latitudes, longitudes):
        print('{}. {}, {}'.format(suburb_id, name, city))
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        if results:
            venues_list.append([(
                                 suburb_id,
                                 name,
                                 postcode,
                                 lga,
                                 city,
                                 lat, 
                                 lng, 
                                 v['venue']['name'], 
                                 v['venue']['location']['lat'], 
                                 v['venue']['location']['lng'],  
                                 v['venue']['categories'][0]['name']
                                ) for v in results])
            
        # for suburbs that didn't return any top venues
        elif not results:
            venues_list.append([(
                                 suburb_id,
                                 name,
                                 postcode,
                                 lga,
                                 city,
                                 lat, 
                                 lng, 
                                 None, 
                                 None, 
                                 None,  
                                 None
                                )])
    
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = [
                             'Suburb_id',
                             'Suburb',
                             'Postcode',
                             'LGA',
                             'City',
                             'Suburb Latitude', 
                             'Suburb Longitude', 
                             'Venue', 
                             'Venue Latitude', 
                             'Venue Longitude', 
                             'Venue Category'
                            ]
    
    return(nearby_venues)

In [23]:
# Explore suburbs
while True:
    try:
        top_venues = getNearbyVenues(suburb_id=aus_suburbs['Suburb_id'],
                                            names=aus_suburbs['Suburb'],
                                            postcode=aus_suburbs['Postcode'],
                                            lga=aus_suburbs['LGA'],
                                            city=aus_suburbs['City'],
                                            latitudes=aus_suburbs['LAT'],
                                            longitudes=aus_suburbs['LON'])
    except(KeyError):
        continue
    break

0. Abbotsbury, Sydney
1. Abbotsford, Sydney
2. Acacia Gardens, Sydney
3. Agnes Banks, Sydney
4. Airds, Sydney
5. Alexandria, Sydney
6. Alfords Point, Sydney
7. Allambie Heights, Sydney
8. Allawah, Sydney
9. Ambarvale, Sydney
10. Annandale, Sydney
11. Annangrove, Sydney
12. Arcadia, Sydney
13. Arncliffe, Sydney
14. Arndell Park, Sydney
15. Artarmon, Sydney
16. Ashbury, Sydney
17. Ashcroft, Sydney
18. Ashfield, Sydney
19. Asquith, Sydney
20. Auburn, Sydney
21. Austral, Sydney
22. Avalon Beach, Sydney
23. Badgerys Creek, Sydney
24. Balgowlah, Sydney
25. Balgowlah Heights, Sydney
26. Balmain, Sydney
27. Balmain East, Sydney
28. Bangor, Sydney
29. Banksia, Sydney
30. Banksmeadow, Sydney
31. Bankstown, Sydney
32. Barangaroo, Sydney
33. Barden Ridge, Sydney
34. Bardia, Sydney
35. Bardwell Park, Sydney
36. Bardwell Valley, Sydney
37. Bass Hill, Sydney
38. Baulkham Hills, Sydney
39. Bayview, Sydney
40. Beacon Hill, Sydney
41. Beaconsfield, Sydney
42. Beaumont Hills, Sydney
43. Beecroft, Sydney


342. Lansdowne, Sydney
343. Lansvale, Sydney
344. Laughtondale, Sydney
345. Lavender Bay, Sydney
346. Leets Vale, Sydney
347. Leichhardt, Sydney
348. Leonay, Sydney
349. Leppington, Sydney
350. Lethbridge Park, Sydney
351. Leumeah, Sydney
352. Lewisham, Sydney
353. Liberty Grove, Sydney
354. Lidcombe, Sydney
355. Lilli Pilli, Sydney
356. Lilyfield, Sydney
357. Lindfield, Sydney
358. Linley Point, Sydney
359. Little Bay, Sydney
360. Liverpool, Sydney
361. Llandilo, Sydney
362. Loftus, Sydney
363. Londonderry, Sydney
364. Long Point, Sydney
365. Longueville, Sydney
366. Lovett Bay, Sydney
367. Lower Portland, Sydney
368. Luddenham, Sydney
369. Lugarno, Sydney
370. Lurnea, Sydney
371. Macquarie Fields, Sydney
372. Macquarie Park, Sydney
373. Malabar, Sydney
374. Manly, Sydney
375. Manly Vale, Sydney
376. Maraylya, Sydney
377. Marayong, Sydney
378. Maroota, Sydney
379. Maroubra, Sydney
380. Marrickville, Sydney
381. Marsden Park, Sydney
382. Marsfield, Sydney
383. Mascot, Sydney
384. Matra

678. Bannockburn, Brisbane
679. Banyo, Brisbane
680. Bardon, Brisbane
681. Barellan Point, Brisbane
682. Basin Pocket, Brisbane
683. Beachmere, Brisbane
684. Beenleigh, Brisbane
685. Belivah, Brisbane
686. Bellara, Brisbane
687. Bellbird Park, Brisbane
688. Bellbowrie, Brisbane
689. Bellmere, Brisbane
690. Belmont, Brisbane
691. Berrinba, Brisbane
692. Bethania, Brisbane
693. Birkdale, Brisbane
694. Blackstone, Brisbane
695. Bongaree, Brisbane
696. Booval, Brisbane
697. Boronia Heights, Brisbane
698. Bowen Hills, Brisbane
699. Bracalba, Brisbane
700. Bracken Ridge, Brisbane
701. Brassall, Brisbane
702. Bray Park, Brisbane
703. Brendale, Brisbane
704. Bridgeman Downs, Brisbane
705. Brighton, Brisbane
706. Brookfield, Brisbane
707. Brookwater, Brisbane
708. Browns Plains, Brisbane
709. Buccan, Brisbane
710. Bulimba, Brisbane
711. Bundamba, Brisbane
712. Bunya, Brisbane
713. Burbank, Brisbane
714. Burpengary, Brisbane
715. Caboolture, Brisbane
716. Caboolture South, Brisbane
717. Calamval

997. Thornlands, Brisbane
998. Tingalpa, Brisbane
999. Tivoli, Brisbane
1000. Toorbul, Brisbane
1001. Underwood, Brisbane
1002. Upper Brookfield, Brisbane
1003. Upper Caboolture, Brisbane
1004. Upper Kedron, Brisbane
1005. Upper Mount Gravatt, Brisbane
1006. Veresdale Scrub, Brisbane
1007. Victoria Point, Brisbane
1008. Virginia, Brisbane
1009. Wacol, Brisbane
1010. Wakerley, Brisbane
1011. Walloon, Brisbane
1012. Wamuran, Brisbane
1013. Wamuran Basin, Brisbane
1014. Warner, Brisbane
1015. Waterford, Brisbane
1016. Waterford West, Brisbane
1017. Wavell Heights, Brisbane
1018. Wellington Point, Brisbane
1019. Welsby, Brisbane
1020. West End, Brisbane
1021. West Ipswich, Brisbane
1022. Westlake, Brisbane
1023. White Patch, Brisbane
1024. Whiteside, Brisbane
1025. Wights Mountain, Brisbane
1026. Willawong, Brisbane
1027. Willowbank, Brisbane
1028. Wilston, Brisbane
1029. Windaroo, Brisbane
1030. Windsor, Brisbane
1031. Wishart, Brisbane
1032. Wolffdene, Brisbane
1033. Woodend, Brisbane
10

1291. Humevale, Melbourne
1292. Huntingdale, Melbourne
1293. Hurstbridge, Melbourne
1294. Iona, Melbourne
1295. Ivanhoe, Melbourne
1296. Ivanhoe East, Melbourne
1297. Jacana, Melbourne
1298. Junction Village, Melbourne
1299. Kalkallo, Melbourne
1300. Kallista, Melbourne
1301. Kalorama, Melbourne
1302. Kangaroo Ground, Melbourne
1303. Kealba, Melbourne
1304. Keilor, Melbourne
1305. Keilor Downs, Melbourne
1306. Keilor East, Melbourne
1307. Keilor Lodge, Melbourne
1308. Keilor North, Melbourne
1309. Keilor Park, Melbourne
1310. Kensington, Melbourne
1311. Kew, Melbourne
1312. Kew East, Melbourne
1313. Keysborough, Melbourne
1314. Kilsyth, Melbourne
1315. Kilsyth South, Melbourne
1316. Kinglake, Melbourne
1317. Kinglake West, Melbourne
1318. Kings Park, Melbourne
1319. Kingsbury, Melbourne
1320. Kingsville, Melbourne
1321. Knoxfield, Melbourne
1322. Koo Wee Rup, Melbourne
1323. Koo Wee Rup North, Melbourne
1324. Kooyong, Melbourne
1325. Kurunjang, Melbourne
1326. Lalor, Melbourne
1327. La

1582. Collinsvale, Hobart
1583. Derwent Park, Hobart
1584. Dodges Ferry, Hobart
1585. Dowsing Point, Hobart
1586. Dynnyrne, Hobart
1587. Fern Tree, Hobart
1588. Gagebrook, Hobart
1589. Geilston Bay, Hobart
1590. Glebe, Hobart
1591. Glenorchy, Hobart
1592. Goodwood, Hobart
1593. Granton, Hobart
1594. Howden, Hobart
1595. Howrah, Hobart
1596. Huntingfield, Hobart
1597. Kingston, Hobart
1598. Kingston Beach, Hobart
1599. Lenah Valley, Hobart
1600. Lindisfarne, Hobart
1601. Lutana, Hobart
1602. Montagu Bay, Hobart
1603. Montrose, Hobart
1604. Moonah, Hobart
1605. Mornington, Hobart
1606. Mount Rumney, Hobart
1607. Mount Stuart, Hobart
1608. New Town, Hobart
1609. North Hobart, Hobart
1610. Oakdowns, Hobart
1611. Old Beach, Hobart
1612. Otago, Hobart
1613. Risdon, Hobart
1614. Risdon Vale, Hobart
1615. Roches Beach, Hobart
1616. Rokeby, Hobart
1617. Rose Bay, Hobart
1618. Rosetta, Hobart
1619. Rosny, Hobart
1620. Rosny Park, Hobart
1621. Sandy Bay, Hobart
1622. Sorell, Hobart
1623. South Ho

1901. Kilburn, Adelaide
1902. Kilkenny, Adelaide
1903. Kings Park, Adelaide
1904. Kingston Park, Adelaide
1905. Kingswood, Adelaide
1906. Klemzig, Adelaide
1907. Kudla, Adelaide
1908. Kurralta Park, Adelaide
1909. Largs Bay, Adelaide
1910. Largs North, Adelaide
1911. Leabrook, Adelaide
1912. Linden Park, Adelaide
1913. Lockleys, Adelaide
1914. Lonsdale, Adelaide
1915. Lower Mitcham, Adelaide
1916. Lynton, Adelaide
1917. Magill, Adelaide
1918. Malvern, Adelaide
1919. Manningham, Adelaide
1920. Mansfield Park, Adelaide
1921. Marden, Adelaide
1922. Marino, Adelaide
1923. Marion, Adelaide
1924. Marleston, Adelaide
1925. Marryatville, Adelaide
1926. Maslin Beach, Adelaide
1927. Mawson Lakes, Adelaide
1928. Maylands, Adelaide
1929. Medindie, Adelaide
1930. Melrose Park, Adelaide
1931. Mile End, Adelaide
1932. Mile End South, Adelaide
1933. Millswood, Adelaide
1934. Mitcham, Adelaide
1935. Mitchell Park, Adelaide
1936. Moana, Adelaide
1937. Modbury, Adelaide
1938. Modbury Heights, Adelaide
19

2206. Tivendale, Darwin
2207. Tiwi, Darwin
2208. Tumbling Waters, Darwin
2209. Virginia, Darwin
2210. Wagait Beach, Darwin
2211. Wagaman, Darwin
2212. Wanguri, Darwin
2213. Weddell, Darwin
2214. Wickham, Darwin
2215. Winnellie, Darwin
2216. Wishart, Darwin
2217. Woodroffe, Darwin
2218. Woolner, Darwin
2219. Wulagi, Darwin
2220. Yarrawonga, Darwin
2221. Zuccoli, Darwin
2222. Alexander Heights, Perth
2223. Alfred Cove, Perth
2224. Alkimos, Perth
2225. Anketell, Perth
2226. Applecross, Perth
2227. Ardross, Perth
2228. Armadale, Perth
2229. Ascot, Perth
2230. Ashby, Perth
2231. Ashfield, Perth
2232. Attadale, Perth
2233. Atwell, Perth
2234. Aubin Grove, Perth
2235. Aveley, Perth
2236. Bailup, Perth
2237. Balcatta, Perth
2238. Baldivis, Perth
2239. Ballajura, Perth
2240. Banjup, Perth
2241. Banksia Grove, Perth
2242. Baskerville, Perth
2243. Bassendean, Perth
2244. Bateman, Perth
2245. Bayswater, Perth
2246. Beaconsfield, Perth
2247. Beckenham, Perth
2248. Bedford, Perth
2249. Bedfordale, P

The number of unique categories that can be curated from all of the returned venues

In [24]:
print('There are {} uniques categories in Sydney.'.format(len(top_venues['Venue Category'].unique())))

There are 457 uniques categories in Sydney.


In [25]:
top_venues.shape

(15925, 11)

In [126]:
top_venues.loc[top_venues['Venue Category'] == 'Zoo Exhibit']

,Suburb_id,Suburb,Postcode,LGA,City,Suburb Latitude,Suburb Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
6607,788,Fig Tree Pocket,4069,"Walter Taylor Ward, City of Brisbane",Brisbane,-27.531716,152.968225,Platypus House,-27.533667,152.968190,Zoo Exhibit
10342,1413,Parkville,3052,"City of Melbourne, City of Moreland",Melbourne,-37.786236,144.947418,Elephant Village,-37.785378,144.950614,Zoo Exhibit
10343,1413,Parkville,3052,"City of Melbourne, City of Moreland",Melbourne,-37.786236,144.947418,Zoos Victoria,-37.786022,144.950260,Zoo Exhibit
10344,1413,Parkville,3052,"City of Melbourne, City of Moreland",Melbourne,-37.786236,144.947418,Trail of the Elephants,-37.784932,144.951598,Zoo Exhibit
10345,1413,Parkville,3052,"City of Melbourne, City of Moreland",Melbourne,-37.786236,144.947418,Gorilla Rainforest,-37.785809,144.951932,Zoo Exhibit
10349,1413,Parkville,3052,"City of Melbourne, City of Moreland",Melbourne,-37.786236,144.947418,Orang-utan Sanctuary,-37.784929,144.951096,Zoo Exhibit
10350,1413,Parkville,3052,"City of Melbourne, City of Moreland",Melbourne,-37.786236,144.947418,Lion Enclosure,-37.783519,144.951184,Zoo Exhibit
10352,1413,Parkville,3052,"City of Melbourne, City of Moreland",Melbourne,-37.786236,144.947418,Australian Bush,-37.784277,144.949647,Zoo Exhibit
10353,1413,Parkville,3052,"City of Melbourne, City of Moreland",Melbourne,-37.786236,144.947418,Great Flight Aviary,-37.784493,144.950329,Zoo Exhibit
10354,1413,Parkville,3052,"City of Melbourne, City of Moreland",Melbourne,-37.786236,144.947418,African Wild Dog Cage,-37.784253,144.951502,Zoo Exhibit


In [128]:
unique_categories = pd.Series.unique(top_venues['Venue Category'])
unique_sorted = sorted(unique_categories, key=lambda x: (x is None, x))

I'll only replace a few categories the time being.

In [130]:
replace_dict = {'Venue Category': {
                                   'Coffee Shop':'Café',
                                   'Modern Greek Restaurant':'Greek Restaurant',
                                   'Rock Climbing Spot':'Climbing Gym'
                                  }
               }

top_venues.replace(replace_dict, inplace=True)

In [131]:
# one hot encoding
top_venues_onehot = pd.get_dummies(top_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
top_venues_onehot['Suburb_id'] = top_venues['Suburb_id'] 
top_venues_onehot['Suburb'] = top_venues['Suburb'] 
top_venues_onehot['Postcode'] = top_venues['Postcode']
top_venues_onehot['LGA'] = top_venues['LGA'] 
top_venues_onehot['City'] = top_venues['City'] 

# move neighborhood column to the first column
fixed_columns = list(top_venues_onehot.columns[-5:]) + list(top_venues_onehot.columns[:-5])
top_venues_onehot = top_venues_onehot[fixed_columns]

top_venues_onehot.head()


,Suburb_id,Suburb,Postcode,LGA,City,ATM,Accessories Store,Adult Boutique,Advertising Agency,Afghan Restaurant,...,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Zoo,Zoo Exhibit
0,0,Abbotsbury,2176,City of Fairfield,Sydney,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,Abbotsford,2046,City of Canada Bay,Sydney,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,Abbotsford,2046,City of Canada Bay,Sydney,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,Abbotsford,2046,City of Canada Bay,Sydney,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,Abbotsford,2046,City of Canada Bay,Sydney,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [132]:
top_venues_onehot.shape

(15925, 458)

Group suburbs and calculate the mean of the frequency of occurance for each category

In [133]:
top_venues_grouped = top_venues_onehot.groupby(['Suburb_id', 'Suburb', 'Postcode', 'LGA', 'City']).sum().reset_index()
top_venues_grouped.head()


,Suburb_id,Suburb,Postcode,LGA,City,ATM,Accessories Store,Adult Boutique,Advertising Agency,Afghan Restaurant,...,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Zoo,Zoo Exhibit
0,0,Abbotsbury,2176,City of Fairfield,Sydney,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,Abbotsford,2046,City of Canada Bay,Sydney,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,2,Acacia Gardens,2763,City of Blacktown,Sydney,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,Agnes Banks,2753,"City of Hawkesbury, City of Penrith",Sydney,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,Airds,2560,City of Campbelltown,Sydney,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Function that sorts most frequent venues in descending order

In [134]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[5:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    top_ten = pd.DataFrame(row_categories_sorted.iloc[0:10])
    top_ten = top_ten.loc[(top_ten!=0).any(axis=1)]
    if len(top_ten) == 0:
        top_ten_list = [None] * num_top_venues
    else:
        top_ten_list = list(top_ten.index.values)
        if len(top_ten_list) < num_top_venues:
            n_empty_cells = num_top_venues - len(top_ten_list)
            top_ten_list.extend(itertools.repeat(None, n_empty_cells))
    
    return top_ten_list

Creating a new dataframe and display the top 10 venues for each neighbourhood

In [135]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Suburb_id', 'Suburb', 'Postcode', 'LGA', 'City']
for index in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(index+1, indicators[index]))
    except:
        columns.append('{}th Most Common Venue'.format(index+1))

# create a new dataframe
top_venues_sorted = pd.DataFrame(columns=columns)
top_venues_sorted['Suburb_id'] = top_venues_grouped['Suburb_id'] 
top_venues_sorted['Suburb'] = top_venues_grouped['Suburb']
top_venues_sorted['Postcode'] = top_venues_grouped['Postcode']
top_venues_sorted['LGA'] = top_venues_grouped['LGA'] 
top_venues_sorted['City'] = top_venues_grouped['City'] 

for index in np.arange(top_venues_grouped.shape[0]):
    top_venues_sorted.iloc[index, 5:] = return_most_common_venues(top_venues_grouped.iloc[index, :], num_top_venues)
    

Save dataframes as csv files

In [136]:
top_venues.to_csv('top_venues.csv')
top_venues_grouped.to_csv('top_venues_grouped.csv')
top_venues_sorted.to_csv('top_venues_sorted.csv')

# Cluster Analysis